In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Filtering out the warnings

import warnings

warnings.filterwarnings('ignore')

# Importing the required libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
movies=pd.read_csv("/kaggle/input/imdb-movie/Movie.csv")
movies.head()

Inspect the dataframe for dimensions, null-values, and summary of different numeric columns.

In [ ]:
# Check the number of rows and columns in the dataframe
movies.shape

In [ ]:
# Check the column-wise info of the dataframe
movies.info()


In [ ]:
# Check the summary for the numeric columns 

movies.describe()

Now that we have loaded the dataset and inspected it, we see that most of the data is in place. As of now, no data cleaning is required, so let's start with some data manipulation, analysis, and visualisation to get various insights about the data.



In [ ]:
# Divide the 'gross' and 'budget' columns by 1000000 to convert '$' to 'million $'

movies.Gross=movies["Gross"].div(1000000)                                  
movies.budget=movies["budget"].div(1000000)
movies.head(12)

In [ ]:
# Create the new column named 'profit' by subtracting the 'budget' column from the 'gross' column
movies["profit"]=movies.Gross-movies.budget
movies.head(10)


In [ ]:
# Sort the dataframe with the 'profit' column as reference using the 'sort_values' function. Make sure to set the argument
#'ascending' to 'False'
movies.sort_values(by=["profit"],ascending=False,inplace=True,ignore_index=True)


In [ ]:
# Get the top 10 profitable movies by using position based indexing. Specify the rows till 10 (0-9)
Top_rated=movies.iloc[0:10,0]
Top_rated

In [ ]:
#Plot profit vs budget
plt.figure(figsize=[20,20])
data=sns.jointplot(data=movies,x="profit",y="budget",color="g")
data.set_axis_labels("Budget","Profit",color="r",fontsize=15)
plt.title("Profit vs Budget",loc="left",fontdict={"fontsize":20,"color":"blue"})
plt.show()


In [ ]:
#Find the movies with negative profit
Negative_Profit_Movies=movies[movies.profit<0]
Negative_Profit_Movies.head(10)

In [ ]:
# Change the scale of MetaCritic

movies.MetaCritic=movies.MetaCritic/10
movies.MetaCritic.head()

In [ ]:
# Find the average ratings
movies["Avg_rating"]=movies.loc[:,["MetaCritic","IMDb_rating"]].mean(axis=1)
#movies["Avg_rating"]=movies.groupby[by='MetaCritic',axis=1].mean()
movies.head(2)

In [ ]:
#Sort in descending order of average rating

movies.sort_values(by=("Avg_rating"),ascending=False,inplace=True)

In [ ]:
# Find the movies with metacritic-Imdb rating < 0.5 and also with an average rating of >= 8 (sorted in descending order)

UniversalAcclaim=movies[(abs(movies.MetaCritic-movies.IMDb_rating)<0.5)]
UniversalAcclaim[(abs(movies.MetaCritic-movies.IMDb_rating)<0.5) & (movies.Avg_rating>=8)]
UniversalAcclaim.sort_values(by=["Avg_rating"],ascending=False,inplace=True,ignore_index=True)
UniversalAcclaim.head()


In [ ]:
# Write your code here
movies["Popular actor"]= movies["actor_1_facebook_likes"]+movies["actor_2_facebook_likes"]+movies["actor_3_facebook_likes"]
Popularity=movies.sort_values(by="Popular actor",ascending=False,ignore_index=True)
five_Popular_Trios=Popularity.loc[:,["actor_1_name","actor_2_name","actor_3_name"]].values.tolist()
five_Popular_Trios[0:5]


In [ ]:
# Runtime histogram/density plot
plt.figure(figsize=[15,5])
sns.histplot(data=movies,x="Runtime",color="green")
plt.title("Movie Runtime Range",fontdict={"fontsize":25,"color":"red"})
plt.xlabel("Runtime",fontdict={"fontsize":15,"color":"blue"})
plt.ylabel("No. of Movies",fontdict={"fontsize":15,"color":"blue"})
plt.show()

Checkpoint 3: Most of the movies appear to be sharply 2 hour-long- More than 25 movies

In [ ]:
# Write your code here
R_Rated=movies[movies["content_rating"]=="R"]
PopularR=R_Rated.sort_values(by="CVotesU18",ascending=False,ignore_index=True)
PopularR[0:5]



Deadpool have been voted the most by people under 18 years

In [ ]:
# Create the dataframe df_by_genre
listofcolumns=[]
for i in movies.columns:
    if i.startswith("CV") or i.startswith("V") or i.startswith("ge"):
        listofcolumns.append(i)
df_by_genre=movies.loc[:,listofcolumns]
df_by_genre.head()

In [ ]:
# Create a column cnt and initialize it to 1

df_by_genre["cnt"]=1
df_by_genre.head()

In [ ]:
# Group the movies by individual genres
Genre1=df_by_genre.groupby("genre_1")
Genre2=df_by_genre.groupby("genre_2")
Genre3=df_by_genre.groupby("genre_3")
df_by_genre.head(5)



In [ ]:
# Add the grouped data frames and store it in a new data frame
Genre1=pd.DataFrame(Genre1.sum())
Genre2=pd.DataFrame(Genre2.sum())
Genre3=pd.DataFrame(Genre3.sum())
Total_data=Genre1.add(Genre2,fill_value=0)
Final_data=Total_data.add(Genre3,fill_value=0)
Final_data.head()


In [ ]:
# Extract genres with atleast 10 occurences
genre_top10=Final_data[Final_data['cnt']>=10]
genre_top10.head()
cnt_column=genre_top10["cnt"]

In [ ]:
# Take the mean for every column by dividing with cnt 

genre_top10=genre_top10.div(genre_top10["cnt"],axis=0)
genre_top10.head()

In [ ]:
# Rounding off the columns of Votes to two decimals
genre_top10=genre_top10.round(2)
genre_top10.head()

In [ ]:
# Converting CVotes to int type
Column_list=[]
for i in genre_top10.columns:
    if i.startswith("CVotes"):
        Column_list.append(i)
genre_top10[Column_list]=genre_top10[Column_list].astype("int")
cnt=cnt_column.tolist()
genre_top10.head()

If you take a look at the final dataframe that you have gotten, you will see that you now have the complete information about all the demographic (Votes- and CVotes-related) columns across the top 10 genres. We can use this dataset to extract exciting insights about the voters!



In [ ]:
# Countplot for genres
plt.figure(figsize=[10,5])
cnt_column=genre_top10["cnt"]
#?sns.barplot
sns.barplot(x=genre_top10.index,y=cnt,color="yellow")
plt.title("Genres vs Count",fontdict={"fontsize":20,"color":"Red"})
plt.xlabel("Genres",fontdict={"fontsize":15,"color":"blue"})
plt.ylabel("Count",fontdict={"fontsize":15,"color":"blue"})
plt.show()

As per the barchart Drama has the highest count

In [ ]:
# 1st set of heat maps for CVotes-related columns
male_votes=genre_top10.groupby(genre_top10.index)["CVotesU18M","CVotes1829M","CVotes3044M","CVotes45AM"].mean()
female_votes=genre_top10.groupby(genre_top10.index)["CVotesU18F","CVotes1829F","CVotes3044F","CVotes45AF"].mean()
f,(ax1,ax2,axcb)=plt.subplots(1,3,gridspec_kw={"width_ratios":[1,1,0.08]})
ax1.get_shared_y_axes().join(ax1,ax2)
male_avg_votes=sns.heatmap(male_votes,cmap="Reds",annot=True,fmt="d",cbar=False,ax=ax1)
male_avg_votes.set_xlabel("Male Age Group",fontsize=10,color="blue")
male_avg_votes.set_ylabel("Genre",fontsize=10,color="blue")
female_avg_votes=sns.heatmap(female_votes,cmap="Greens",annot=True,fmt="d",cbar_ax=axcb,ax=ax2)
female_avg_votes.set_xlabel("Female Age Group",fontdict={"fontsize":10,"color":"blue"})
plt.ylabel(" ")
female_avg_votes.set_yticks([])
for ax in [male_avg_votes,female_avg_votes]:
    t1=ax.get_xticklabels()
    ax.set_xticklabels(t1, rotation=90)
    tly=ax.get_yticklabels()
    ax.set_yticklabels(tly, rotation=0)

A few inferences that can be seen from the heatmap above is that males have voted more than females, and Sci-Fi appears to be most popular among the 18-29 age group irrespective of their gender. What more can you infer from the two heatmaps that you have plotted? Write your three inferences/observations below:

Inference 1: Age group 18 to 44 in both male and female have voted more
Inference 2: Adventure movies are watched more by both the genders in the age group of 18 to 29
Inference 3: Age less than 18 and above 45 have watched less movies irrespective of the gender

In [ ]:
# 2nd set of heat maps for Votes-related columns
male_votes=genre_top10.groupby(genre_top10.index)["VotesU18M","Votes1829M","Votes3044M","Votes45AM"].mean()
female_votes=genre_top10.groupby(genre_top10.index)["VotesU18F","Votes1829F","Votes3044F","Votes45AF"].mean()
f,(ax1,ax2,axcb)=plt.subplots(1,3,gridspec_kw={"width_ratios":[1,1,0.08]})
ax1.get_shared_y_axes().join(ax1,ax2)
male_avg_votes=sns.heatmap(male_votes,cmap="Reds",annot=True,cbar=False,ax=ax1)
male_avg_votes.set_xlabel("Male Age Group",fontsize=10,color="blue")
male_avg_votes.set_ylabel("Genre",fontsize=10,color="blue")
female_avg_votes=sns.heatmap(female_votes,cmap="Greens",annot=True,cbar_ax=axcb,ax=ax2)
female_avg_votes.set_xlabel("Female Age Group",fontdict={"fontsize":10,"color":"blue"})
plt.ylabel(" ")
female_avg_votes.set_yticks([])
for ax in [male_avg_votes,female_avg_votes]:
    t1=ax.get_xticklabels()
    ax.set_xticklabels(t1, rotation=90)
    tly=ax.get_yticklabels()
    ax.set_yticklabels(tly, rotation=0)

Sci-Fi appears to be the highest rated genre in the age group of U18 for both males and females. Also, females in this age group have rated it a bit higher than the males in the same age group. What more can you infer from the two heatmaps that you have plotted? Write your three inferences/observations below:

Inference 1: Age above 45 have rated all the movie generes less than age in both the genders
Inference 2:Average of drama rating rating is very similar for all the age groups for both the genders
Inference 3: Theres a significant difference based on the ratings of animation compared to both the genders

In [ ]:
# Creating IFUS column
movies["IFUS"]=movies["Country"].apply(lambda x:"USA" if x=="USA" else "non-USA")
movies.head()


In [ ]:
# Box plot - 1: CVotesUS(y) vs IFUS(x)
plt.figure(figsize=[25,25])
fig,axes=plt.subplots(nrows=1,ncols=2)
USA_Movies=sns.boxplot(data=movies,x="IFUS",y="CVotesUS",ax=axes[0])
USA_Movies.axes.set_title("USA People Votes",fontsize=10,color="red")
USA_Movies.set_xlabel("IFUS",fontsize=10)
USA_Movies.set_ylabel("CVotesUS",fontsize=10)
Non_USA_Movies=sns.boxplot(data=movies,x="IFUS",y="CVotesnUS",ax=axes[1])
Non_USA_Movies.axes.set_title("Non-USA People Votes",fontsize=10,color="red")
Non_USA_Movies.set_xlabel("IFUS",fontsize=10)
Non_USA_Movies.set_ylabel("CVotesnUS",fontsize=10)
plt.tight_layout()
plt.show()


Inference 1: Outliers are visible for USA movies in both the plots
Inference 2: Overall votes for USA movies are higher form both USA and NON USA people

In [ ]:
# Box plot - 2: VotesUS(y) vs IFUS(x)
plt.figure(figsize=[25,25])
fig,axes=plt.subplots(nrows=1,ncols=2)
USA_Movies=sns.boxplot(data=movies,x="IFUS",y="VotesUS",ax=axes[0])
USA_Movies.axes.set_title("USA People Votes",fontsize=10,color="red")
USA_Movies.set_xlabel("IFUS",fontsize=10)
USA_Movies.set_ylabel("CVotesUS",fontsize=10)
Non_USA_Movies=sns.boxplot(data=movies,x="IFUS",y="VotesnUS",ax=axes[1])
Non_USA_Movies.axes.set_title("Non-USA People Votes",fontsize=10,color="red")
Non_USA_Movies.set_xlabel("IFUS",fontsize=10)
Non_USA_Movies.set_ylabel("CVotesnUS",fontsize=10)
plt.tight_layout()
plt.show()


Inference 1: Outliers are visible in both the plots for USA Movies though it is higher in Non-USA People Votes
Inference 2: Both USA and Non-USA people have voted more for USA movies


In [ ]:
# Sorting by CVotes1000
genre_CVotes1000=genre_top10.sort_values(by="CVotes1000",ascending=False)
genre_CVotes1000

In [ ]:
# Bar plot
plt.figure(figsize=[10,5])
sns.barplot(x=genre_CVotes1000.index,y=genre_CVotes1000.CVotes1000,color="yellow")
plt.title("Genre Vs CVotes",fontsize=20)
plt.xlabel("Genre")
plt.show()

Sci-fi is voted the highest
Romance is voted the lowest of all
Theres very less different between Action, Thriller and Adventure but Crime has significant difference compared to these three

Check Point - The genre Romance seems to be most unpopular among the top 1000 voters.

As per the bar chart that have been voted less compared to the other genres